### 수행목표
- 랜덤 추천의 결과를 통해 분류 평가를 계산하는 기능을 개발한다.
### 수행단계
- 기존 rate_random은 그대로 두고 rate_random_class 필드를 추가해서 랜덤 평점을 반올림한 값을 업데이트 한다.
- 아래와 같은 평가 계산을 Analyzer class에 추가한다.
    - 혼동 행렬 (Confusion Matrix)을 생성한다.
    - 정확도 (Accuracy)를 계산한다.
    - 정밀도 (Precision)를 계산한다.
    - 재현율 (Recall)을 계산한다.
    - F1 점수 (F1 Score)를 계산한다.  
- 결과 데이터 평점 높은 순 n개를 출력하고 그 아래에 분석 결과를 출력한다.


In [ ]:
# 혼동 행렬 (Confusion Matrix): 실제 값과 예측 값의 일치 여부를 시각적으로 표현한 행렬
# 정확도 (Accuracy): 전체 데이터 중 올바르게 예측된 비율
# 정밀도 (Precision): 양성으로 예측한 데이터 중 실제로 양성인 비율
# 재현율 (Recall): 실제 양성 중에서 모델이 올바르게 예측한 비율
# F1 점수 (F1 Score): 정밀도와 재현율의 조화 평균

In [88]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [89]:
# 데이터를 load() 하고 전처리하는 class
class MovieDataLoader:
    def __init__(self, movies_file: str, rates_file: str):
        # 영화 데이터 파일 경로와 평점 데이터 파일 경로를 저장
        self.movies_file = movies_file
        self.rates_file = rates_file

    def load(self):
        # 영화 데이터(movies.txt) 파일을 로드
        movies = pd.read_csv(self.movies_file, sep="\t", engine="python", index_col="movie")
        
        # 평점 데이터(rates.csv) 파일을 로드
        rates = pd.read_csv(self.rates_file)
        
        # 로드된 데이터프레임 반환
        return movies, rates

In [90]:
# 랜덤 추천 및 추가 필드 업데이트를 담당하는 class
class RandomRecommender:
    def __init__(self, movies, rates):
        # 생성자에서 영화 데이터와 평점 데이터를 받아서 저장
        self.movies = movies
        self.rates = rates

    def run(self, n=10):
        # 1. 랜덤 평점(rate_random) 생성
        # 아래에서 일의 자리 반올림하기 위해 기존에 작성한 np.round는 삭제함 (여기까지 실수값)
        self.rates['rate_random'] = np.random.uniform(0, 10, size=len(self.rates))
        
        # 2. 랜덤 평점을 반올림한 값으로 rate_random_class 필드 추가
        # 반올림하여 정수로 변환 (실수를 1의 자리까지 반올림 -> int로 정수화, astype(int)는 버림 처리되므로 .round() 없이 사용하면 수행단계에서 의도한 반올림 처리가 안됨)
        self.rates['rate_random_class'] = self.rates['rate_random'].round().astype(int)
        
        print(self.rates['rate_random_class'].unique())

        # 3. 영화 데이터와 평점 데이터를 병합
        merged_data = pd.merge(self.rates, self.movies, left_on='movie', right_index=True)
        
        # 4. 랜덤 평점(rate_random) 기준으로 내림차순 정렬
        sorted_data = merged_data.sort_values(by="rate_random", ascending=False)
        
        # 상위 n개의 데이터 반환
        return sorted_data.head(n)

In [93]:
# 결과를 분석하는 class
class Analyzer:
    def __init__(self, result_data):
        # 분석할 데이터 저장
        self.result_data = result_data

    def calculate_metrics(self):
        # 1. 실제 평점(rate)과 랜덤 평점 클래스(rate_random_class)를 사용하여 혼동 행렬 생성
        y_true = self.result_data['rate']  # 실제 평점
        y_pred = self.result_data['rate_random_class']  # 랜덤 평점 클래스
        
        print("y_true(실제 평점) 고유 값:", set(y_true))
        print("y_pred(랜덤 평점) 고유 값:", set(y_pred))
        print("rate_random_class 값 분포:", self.result_data['rate_random_class'].value_counts())

        # 혼동 행렬
        conf_matrix = confusion_matrix(y_true, y_pred)
        
        # 2. 평가 지표 계산
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

        # 결과 반환
        return {
            "confusion_matrix": conf_matrix,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        }

In [106]:
# 메인 실행부
if __name__ == "__main__":
    # 영화 데이터 파일 경로
    movies_file = "kmrd-small/movies.txt"
    # 평점 데이터 파일 경로
    rates_file = "kmrd-small/rates.csv"

    # 데이터 로드
    loader = MovieDataLoader(movies_file, rates_file)
    movies, rates = loader.load()

    # 랜덤 추천 실행
    recommender = RandomRecommender(movies, rates)
    top_recommendations = recommender.run(n=3)

    # 추천 결과 출력
    print("-------------------------")
    print("----- 추천 결과 -----")
    print("-------------------------")
    result = top_recommendations[['rate_random', 'rate_random_class', 'movie', 'rate', 'title', 'year']]
    print(result)

    # 결과 분석
    analyzer = Analyzer(result)
    metrics = analyzer.calculate_metrics()


# 분석 결과 출력
print()
print("-------------------------")
print("----- 분석 결과 -----")
print("-------------------------")

# 혼동 행렬
print("Confusion Matrix (혼동 행렬):")
print(metrics["confusion_matrix"])
print(f"Accuracy (정확도): {metrics['accuracy']:.2f}")
print(f"Precision (정밀도): {metrics['precision']:.2f}")
print(f"Recall (재현율): {metrics['recall']:.2f}")
print(f"F1 Score (F1 점수): {metrics['f1_score']:.2f}")


[ 4  8  6  3  9  7  5  1 10  0  2]
-------------------------
----- 추천 결과 -----
-------------------------
        rate_random  rate_random_class  movie  rate    title    year
42502      9.999974                 10  10072     1     대부 2  2010.0
43564      9.999973                 10  10238    10     레인 맨  1989.0
136653     9.999883                 10  10746    10  폴리스 스토리  2015.0
y_true(실제 평점) 고유 값: {1, 10}
y_pred(랜덤 평점) 고유 값: {10}
rate_random_class 값 분포: rate_random_class
10    3
Name: count, dtype: int64

-------------------------
----- 분석 결과 -----
-------------------------
Confusion Matrix (혼동 행렬):
[[0 1]
 [0 2]]
Accuracy (정확도): 0.67
Precision (정밀도): 0.44
Recall (재현율): 0.67
F1 Score (F1 점수): 0.53
